In [8]:
# Environment setup and module import
from __future__ import print_function
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder


In [9]:
# Import DF from CSV

#train_file = 'data/train_clean.csv'
train_file = 'data/mini_train.csv'
train_df = pd.read_csv(train_file)


In [10]:
# split out labels and ids
train_labels = train_df["HasDetections"].values
train_ids = train_df["MachineIdentifier"].values
train_df = train_df.drop(columns=["HasDetections", "MachineIdentifier"])


In [11]:
# Perform whatever conversions are necessary to get everything into proper format: ndarray

le = preprocessing.LabelEncoder()
train2 = train_df[train_df.select_dtypes(object).columns].apply(le.fit_transform)
for c in train_df[train_df.select_dtypes(int).columns]:
    train2[c] = train_df[c]
for c in train_df[train_df.select_dtypes(bool).columns]:
    train2[c] = train_df[c]

# To do: Perform standardization for input parameters

x_train = train2
y_train = train_labels


In [12]:
# Testing Cell
print(x_train.shape)
print(y_train.shape)

(8921483, 104)
(8921483,)


In [13]:
#
# save
#

#train_file = 'data/train_encoded.csv'
train_file = 'data/mini_train_encoded.csv'
train_df.to_csv(train_file)
